In [13]:
import tensorflow as tf
import keras
from IPython.display import clear_output
import matplotlib.pyplot as plt
# check GPU available
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.config.list_physical_devices('GPU')

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3281632008735706307
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6472244177088730245
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7575883907298277908
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11758386560
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2341130766530777370
physical_device_desc: "device: 0, name: TITAN Xp, pci bus id: 0000:65:00.0, compute capability: 6.1"
]


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [14]:
import numpy as np
# load in both train and val data paths
train_path = np.load('/BrainSeg/data/patches_1024/train.npy')
val_path = np.load('/BrainSeg/data/patches_1024/val.npy')

In [15]:
# upsample function utilizing tf.keras.layers.Conv2D, size=4, strides=2, default set to be 2x resolution
# based on reference http://warmspringwinds.github.io/tensorflow/tf-slim/2016/11/22/upsampling-and-image-segmentation-with-tensorflow-and-tf-slim/
# upsample function, the size is determined by factor of images, strides is 2 * factor - factor % 2
def upsample(filters, size=4, strides=2, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=strides,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

    result.add(tf.keras.layers.BatchNormalization())

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())

    return result

In [16]:
def fcn_model(classes=3, drop_out_rate=0.2, bn=True):
    # use dropout and bacth normalization to prevent overfitting and help to do quick convergence
    # activation layer is added to incorporate non-linearity
    
    # input layer has variable length in width and height, tested on both 512, 1024
    input_imgs = tf.keras.layers.Input(shape=(None, None, 3))
    
    # All the kernel size, filter, stride are based on comparson paper, maxpooling layer is based on original FCN paper
    
    # First conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=1, padding='same')(input_imgs)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    x = tf.keras.layers.Activation('relu')(x)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)
    
    # Second conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=1, padding='same')(pool1)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    x = tf.keras.layers.Activation('relu')(x)
    
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)
    
    # Third conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(pool2)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    x = tf.keras.layers.Activation('relu')(x)
    
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)
    
    # Forth conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding="same")(pool3)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    x = tf.keras.layers.Activation('relu')(x)
    
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)  
    
    # Fifth conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=1024, kernel_size=11, strides=1, padding="same")(pool4)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    x = tf.keras.layers.Activation('relu')(x)
    
    pool5 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)
    
    # build the fully connected layer using 1*1 convolutional layer
    x = tf.keras.layers.Conv2D(filters=512, kernel_size=1, strides=1, padding="same")(pool5)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    conv6 = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Conv2D(filters=classes, kernel_size=1, strides=1, padding="same")(conv6)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    conv7 = tf.keras.layers.Activation('sigmoid')(x)

    # upsampling conv7 to 4x times and upsample pool4 to 2x times
    up_conv7 = upsample(filters=classes, size=8, strides=4)(conv7)
    up_pool4 = upsample(filters=64)(pool4)
    
    # Concatenate two resolutions
    fuse_1 = tf.keras.layers.Concatenate()([up_conv7, up_pool4])
    fuse_2 = tf.keras.layers.Concatenate()([fuse_1, pool3])
    
    prob = upsample(filters=classes, size=16, strides=8)(fuse_2)
    model = tf.keras.Model(inputs=input_imgs, outputs=prob)
    
    print(model.summary())
    print("FCN model building completes")
    
    return model

In [17]:
# total number of output being 3 masks (white, grey, background)
# this loss functon is recommended for segmentation masks, set batch normalization to be training true
model = fcn_model(classes=3)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[ tf.keras.metrics.SparseCategoricalAccuracy()])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, None, None, 1 1216        input_2[0][0]                    
__________________________________________________________________________________________________
dropout_7 (Dropout)             (None, None, None, 1 0           conv2d_7[0][0]                   
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, None, None, 1 64          dropout_7[0][0]                  
____________________________________________________________________________________________

In [18]:
# Dataset class structure, combine with numpy array color normalization
from tensorflow.keras.utils import Sequence
from typing import Tuple
from nptyping import NDArray
from PIL import Image
class BrainSegSequence(Sequence):
    def __init__(self, image_paths: NDArray[str],
            mask_paths: NDArray[str], batch_size: int):
        self.image_paths = image_paths
        self.mask_paths  = mask_paths
        self.batch_size  = batch_size

    def __len__(self) -> int:
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, idx: int) -> Tuple[NDArray[np.uint8], NDArray[np.uint8]]:
        batch_x = self.image_paths[idx * self.batch_size : 
                (idx+1) * self.batch_size]
        batch_y = self.mask_paths[idx * self.batch_size : 
                (idx+1) * self.batch_size]
        return np.array([np.array(Image.open(p)) for p in batch_x])/255.0, \
                np.array([np.array(Image.open(p)) for p in batch_y])

In [19]:
# Construct the training and val dataset with batchsize 16
BATCH_SIZE = 16
train_dataset = BrainSegSequence(train_path[:,0], train_path[:,1], BATCH_SIZE)
val_dataset = BrainSegSequence(val_path[:,0], val_path[:,1], BATCH_SIZE)

In [21]:
# return the latest weights in the folder
latest = tf.train.latest_checkpoint('/BrainSeg/baseline_code/5_3_saved_weights/')
# Load the previously saved weights
model.load_weights(latest)
latest

'/BrainSeg/baseline_code/5_3_saved_weights/cp-0005.ckpt'

In [22]:
import os
checkpoint_path = "./5_4_saved_weights/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=1)
# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

In [24]:
EPOCHS = 12
VAL_SUBSPLITS = 5
STEPS_PER_EPOCH = len(train_path) // BATCH_SIZE
VALIDATION_STEPS = len(val_path)//BATCH_SIZE//VAL_SUBSPLITS
model_history = model.fit(train_dataset, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=val_dataset,
                          callbacks=[cp_callback]
                          )

Epoch 1/12
 153/2861 [>.............................] - ETA: 46:14 - loss: 0.1109 - sparse_categorical_accuracy: 0.9649

KeyboardInterrupt: 